In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from amberdata_derivatives import AmberdataDerivatives
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import mplfinance as mpf
import numpy as np
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# insert API key below
amberdata_client = AmberdataDerivatives(api_key="")

amberdata_client.get

In [ ]:
btc = pd.DataFrame(amberdata_client.get_volatility_term_structures_constant(currency='BTC', exchange='deribit')['payload']['data'])
eth = pd.DataFrame(amberdata_client.get_volatility_term_structures_constant(currency='ETH', exchange='deribit')['payload']['data'])

# plot the atm BTC + ETH term structure
plt.figure(figsize=(15, 7))
plt.plot(eth.daysToExpiration, eth.atm, label='ETH')
plt.plot(btc.daysToExpiration, btc.atm, label='BTC')
plt.title("ATM Term Structure Comparison BTC vs. ETH as of " + str(str(pd.to_datetime(btc.timestamp[0]))))
plt.ylabel("Implied Volatility (%)")
plt.xlabel("DTE")
plt.legend()

# plot BTC atm + atm forward vol
plt.figure(figsize=(15, 7))
plt.plot(btc.daysToExpiration, btc.atm, label='Mark')
plt.plot(btc.daysToExpiration, btc.fwdAtm, label='Forward', linestyle='--')
plt.title("ATM Term Structure Comparison BTC Mark vs Forward Curve " + str(str(pd.to_datetime(btc.timestamp[0]))))
plt.ylabel("Implied Volatility (%)")
plt.xlabel("DTE")
plt.legend();

In [ ]:
# plot atm BTC volatility
data = amberdata_client.get_volatility_delta_surfaces_constant(currency='BTC', exchange='deribit', startDate='2024-01-01', endDate='2024-06-01', timeInterval='day')
data = data['payload']['data']
data = pd.DataFrame(data)

date = pd.to_datetime(data[data.daysToExpiration==7].timestamp)

plt.figure(figsize=(15, 7))
plt.plot(date, data[data.daysToExpiration==7].atm, label='7 DTE')
plt.plot(date, data[data.daysToExpiration==30].atm, label='30 DTE')
plt.plot(date, data[data.daysToExpiration==60].atm, label='60 DTE')
plt.plot(date, data[data.daysToExpiration==180].atm, label='180 DTE')
plt.legend()
plt.title("BTC ATM Volatility: 2024-01-01 to 2024-06-01")
plt.ylabel("ATM IV (%)");

In [ ]:
# plot 25 delta BTC skew
data = amberdata_client.get_volatility_delta_surfaces_constant(currency='BTC', exchange='deribit', startDate='2019-04-01', endDate='2022-06-01', timeInterval='day')
data = data['payload']['data']
data = pd.DataFrame(data)

s7 = data[data.daysToExpiration==7]
s30 = data[data.daysToExpiration==30]
s90 = data[data.daysToExpiration==90]

plt.figure(figsize=(15, 7))
plt.plot(s7.deltaCall10 / s7.deltaCall25, label='7 DTE ∆10/∆25')
plt.plot(s30.deltaCall10 / s30.deltaCall25, label='30 DTE ∆10/∆25')
plt.plot(s90.deltaCall10 / s90.deltaCall25, label='90 DTE ∆10/∆25')
plt.axhline(1, linestyle='--', c='r', alpha=0.50)
plt.legend()
plt.title("BTC ∆25/∆35 Ratio [Call - Put]: 2022-01-01 to 2024-06-01")
plt.ylabel("Skew [Call - Put] (%)")

In [ ]:
# plot BTC Deribit vol curves 
data = pd.DataFrame(amberdata_client.get_volatility_level_1_quotes(exchange='deribit', currency='btc')['payload']['data'])
mats = sorted(set(data.expirationTimestamp))

plt.figure(figsize=(15, 7))
for mat in mats:
    sub = data[data.expirationTimestamp == mat]
    sub = sub.sort_values("strike").reset_index(drop=True)
    
    plt.plot(sub.strike, sub.markIv, label=mat[:10])
    plt.legend()
    plt.title("BTC Volatility Curves as of " + str(data.timestamp[0]))
    plt.ylabel("Implied Volatility (%)")
    plt.xlabel("Strike ($)")

In [ ]:
data = amberdata_client.get_volatility_term_structures_richness(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp)

plt.figure(figsize=(15, 7))
plt.plot(data.richness)
plt.title("BTC Term Structure Richness: 2024-01-01 to 2024-05-01")
plt.ylabel("Term Structure Richness Ratio")

In [ ]:
# plot BTC DVOL 
dvol = amberdata_client.get_volatility_index(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-06-01', timeInterval='day')
dvol = pd.DataFrame(dvol['payload']['data'])
dvol.index = pd.to_datetime(dvol.exchangeTimestamp, unit='ms')
dvol = dvol[dvol.currency=='BTC']
dvol = dvol.sort_index()

mpf.plot(dvol, type='candle', style='charles', figratio=(15, 7), 
         ylabel='DVOL Index', axtitle='BTC DVOL: 2024-01-01 to 2024-06-01', tight_layout=True)

In [ ]:
# plot DVOL vs spot and RR vs spot
data = amberdata_client.get_volatility_index_decorated(exchange='deribit', currency='btc',  startDate='2024-01-01', endDate='2024-06-01', timeInterval='day')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.exchangeTimestamp, unit='ms')
data = data[data.currency=='BTC']
data = data.sort_index()

plt.figure(figsize=(15, 7))
scatter = plt.scatter(data.indexPrice, data.atm, c=data.daysAgo, cmap='coolwarm')
cbar = plt.colorbar(scatter)
cbar.set_label('Days Ago')
plt.title("BTC ATM IV & Spot: 2024-01-01 to 2024-06-01")
plt.xlabel("Spot Price")
plt.ylabel("ATM IV")

In [ ]:
plt.figure(figsize=(15, 7))
scatter = plt.scatter(data.indexPrice, data.delta25RrSkew, c=data.daysAgo, cmap='coolwarm')
cbar = plt.colorbar(scatter)
cbar.set_label('Days Ago')
plt.title("BTC 25 Delta RR Skew & Spot: 2024-01-01 to 2024-06-01")
plt.xlabel("Spot Price")
plt.ylabel("25 RR Skew")

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(data.close/data.atm)
plt.title("BTC DVOL / ATM 30 DTE IV: 2024-01-01 to 2024-06-01")
plt.ylabel("Butterfly Ratio")

In [ ]:
# get vol metrics table
data = amberdata_client.get_volatility_metrics(exchange='deribit', currency='btc', daysBack=7)
data = pd.DataFrame(data['payload']['data'])
data